In [1]:
import cv2
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.compat.v1.reset_default_graph()

IMG_H_SIZE = 32
IMG_W_SIZE = 32

2023-06-29 18:39:21.428097: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 18:39:21.508479: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 18:39:21.509296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 18:39:22.687532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Funcion para leer imágenes

In [2]:
def read_images(dirname):
    imgpath = dirname + os.sep
    images = []
    directories = []
    dircount = []
    prevRoot=''
    cant=0
    print("leyendo imagenes de ",imgpath)

    for root, dirnames, filenames in os.walk(imgpath):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                cant+=1
                filepath = os.path.join(root, filename)
                # image = plt.imread(filepath)
                image = cv2.imread(filepath)
                image = cv2.resize(image, (IMG_H_SIZE, IMG_W_SIZE))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #Función de Normalización
                # image = cv2.normalize(image, None, alpha=0,beta=200, norm_type=cv2.NORM_MINMAX)
                images.append(image)
                if prevRoot !=root:
                    prevRoot=root
                    directories.append(root)
                    dircount.append(cant)
                    cant=0
    dircount.append(cant)

    dircount = dircount[1:]
    dircount[0]=dircount[0]+1
    print('Directorios leidos:',len(directories))
    print("Imagenes en cada directorio", dircount)
    print('Suma Total de imagenes en subdirs:',sum(dircount))

    tipos=[]
    indice=0
    for directorio in directories:
        name = directorio.split(os.sep)
        print(indice , name[len(name)-1])
        tipos.append(name[len(name)-1])
        indice=indice+1

    labels=[]
    indice=0
    for cantidad in dircount:
        for i in range(cantidad):
            labels.append(tipos[indice])
        indice=indice+1

    X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
    y = np.array(labels)
    return X, y

Creamos Sets de Entrenamiento y Test
- En la función read_images, normalizamos cada imagen debido a que se encontró varias imágenes con un nivel de intensidad alto. Se utilizó esta ténica para obtener un mejor contraste en las imágenes con poco contraste debido al brillo.

In [3]:
X_train,y_train = read_images(os.path.join(os.getcwd(), 'CarneDataset/train'))
X_test,y_test = read_images(os.path.join(os.getcwd(), 'CarneDataset/test'))

leyendo imagenes de  /home/jhonatan/Documentos/UIDE/TRATAMIENT/Clasificacion_Imagenes/CarneDataset/train/
Directorios leidos: 7
Imagenes en cada directorio [64, 213, 62, 204, 105, 37, 948]
Suma Total de imagenes en subdirs: 1633
0 CLASS_08
1 CLASS_03
2 CLASS_02
3 CLASS_07
4 CLASS_04
5 CLASS_06
6 CLASS_05
leyendo imagenes de  /home/jhonatan/Documentos/UIDE/TRATAMIENT/Clasificacion_Imagenes/CarneDataset/test/
Directorios leidos: 8
Imagenes en cada directorio [28, 97, 48, 114, 45, 1, 19, 458]
Suma Total de imagenes en subdirs: 810
0 CLASS_08
1 CLASS_03
2 CLASS_02
3 CLASS_07
4 CLASS_04
5 CLASS_01
6 CLASS_06
7 CLASS_05


Prepocesar las Imagenes - Normalizamos las imágenes de entrenamiento

In [4]:
X_train = X_train.astype('float32')
X_train = X_train / 255.0

Codificamos las etiquetas como enteros

In [5]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

Hacemos el One-hot Encoding para la red

In [6]:
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

Creamos el Set de Entrenamiento y Validación

In [7]:
train_X,valid_X,train_label,valid_label = train_test_split(X_train, train_Y_one_hot, test_size=0.2, random_state=13)
classes_train = np.unique(y_train)
nClasses_train = len(classes_train)


Creamos el modelo de CNN

In [8]:
INIT_LR = 1e-3 # Valor inicial de learning rate. El valor 1e-3 corresponde con 0.001
epochs = 50 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 200 # Cantidad de imágenes que se toman a la vez en memoria
classifier_model = Sequential()
classifier_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(IMG_W_SIZE,IMG_H_SIZE,3)))
classifier_model.add(LeakyReLU(alpha=0.1))
classifier_model.add(MaxPooling2D((2, 2),padding='same'))
classifier_model.add(Dropout(0.5))

classifier_model.add(Flatten())
classifier_model.add(Dense(32, activation='linear'))
classifier_model.add(LeakyReLU(alpha=0.1))
classifier_model.add(Dropout(0.5))
classifier_model.add(Dense(nClasses_train, activation='softmax'))
classifier_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 32)                262176    
                                                        

2023-06-29 18:40:12.016627: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Para que nuestro modelo sea más robusto utilizaremos ImageDataGenerator de tf.keras para realizar modificaciones sencillas a las imágenes de entrenamiento. En este caso, vamos a rotar las imágenes, añadir (o quitar) zoom, cambiar el brillo y la intensidad de los canales, e inclinarlas.

In [9]:
datagen = ImageDataGenerator(rotation_range=10,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.1,
                             shear_range=0.1,
                             channel_shift_range=0.1,
                             brightness_range=[0.95, 1.05])

Entrenamos el modelo CNN

In [10]:
classifier_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])
# classifier_train = classifier_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

classifier_train = classifier_model.fit(datagen.flow(train_X, train_label, batch_size=batch_size),epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

/home/jhonatan/Documentos/UIDE/TRATAMIENT/Clasificacion_Imagenes/ven/lib/python3.8/site-packages/keras/optimizers/legacy/adagrad.py:84: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50


2023-06-29 18:40:21.320401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 3s 308ms/step - loss: 1.9404 - accuracy: 0.4579 - val_loss: 1.8324 - val_accuracy: 0.5749
Epoch 2/50
7/7 [==============================] - 2s 273ms/step - loss: 1.9346 - accuracy: 0.5061 - val_loss: 1.8227 - val_accuracy: 0.5780
Epoch 3/50
7/7 [==============================] - 2s 269ms/step - loss: 1.9266 - accuracy: 0.5092 - val_loss: 1.8125 - val_accuracy: 0.5841
Epoch 4/50
7/7 [==============================] - 2s 257ms/step - loss: 1.9195 - accuracy: 0.4793 - val_loss: 1.8010 - val_accuracy: 0.5719
Epoch 5/50
7/7 [==============================] - 2s 246ms/step - loss: 1.9147 - accuracy: 0.4939 - val_loss: 1.7885 - val_accuracy: 0.5688
Epoch 6/50
7/7 [==============================] - 2s 233ms/step - loss: 1.9066 - accuracy: 0.4977 - val_loss: 1.7744 - val_accuracy: 0.5719
Epoch 7/50
7/7 [==============================] - 2s 232ms/step - loss: 1.9028 - accuracy: 0.4923 - val_loss: 1.7594 - val_accuracy: 0.5749
Epoch 8/50
7/7 [===============

Predicción para Test

In [11]:
predicted_classes_result = classifier_model.predict(X_test)

predicted_classes_test=[]
for predicted_image in predicted_classes_result:
    predicted_classes_test.append(predicted_image.tolist().index(max(predicted_image)))
predicted_claspredicted_classes_testses=np.array(predicted_classes_test)

correct_test = np.where(predicted_classes_test==y_test)[0]

print ('Correct test values %s from %s'%(len(correct_test),len(y_test)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_test)/len(y_test)*100))
confusion_matrix(y_test, predicted_classes_test)

26/26 [==============================] - 0s 6ms/step
Correct test values 45 from 810
Porcentaje valores correctos: 5.56


array([[  0,   0,   0,   1,   0,   0,   0,   0],
       [  0,   0,   0,  48,   0,   0,   0,   0],
       [  0,   0,   0,  97,   0,   0,   0,   0],
       [  0,   0,   0,  45,   0,   0,   0,   0],
       [  0,   0,   0, 458,   0,   0,   0,   0],
       [  0,   0,   0,  19,   0,   0,   0,   0],
       [  0,   0,   0, 114,   0,   0,   0,   0],
       [  0,   0,   0,  28,   0,   0,   0,   0]])

Predicción para Training

In [12]:
predicted_classes_result = classifier_model.predict(X_train)

predicted_classes_train=[]
for predicted_image in predicted_classes_result:
    predicted_classes_train.append(predicted_image.tolist().index(max(predicted_image)))
predicted_classes_train=np.array(predicted_classes_train)

correct_train = np.where(predicted_classes_train==y_train)[0]

print ('Correct train values %s from %s'%(len(correct_train),len(y_train)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_train)/len(y_train)*100))
confusion_matrix(y_train, predicted_classes_train)

52/52 [==============================] - 0s 6ms/step
Correct train values 948 from 1633
Porcentaje valores correctos: 58.05


array([[  0,   0,   0,  62,   0,   0,   0],
       [  0,   0,   0, 213,   0,   0,   0],
       [  0,   0,   0, 105,   0,   0,   0],
       [  0,   0,   0, 948,   0,   0,   0],
       [  0,   0,   0,  37,   0,   0,   0],
       [  0,   0,   0, 204,   0,   0,   0],
       [  0,   0,   0,  64,   0,   0,   0]])